![Colegio Bourbaki](./Images/Bourbaki.png)

##  Rudimentos de Machine Learning

### Contexto

NP Paribas Personal Finance es líder en financiación personal en Francia y en Europa a través de su actividad de crédito al consumo. Filial al 100% del Grupo BNP Paribas, BNP Paribas Personal Finance reúne a más de 20 000 empleados y opera en una treintena de países. Bajo diversas marcas como Cetelem, Cofinoga y Findomestic, BNP Paribas Personal Finance ofrece a sus clientes una gama completa de créditos al consumo, disponibles en tiendas y concesionarios de automóviles o directamente a través de centros de relación con el cliente y de los sitios web locales de la empresa.

BNP Paribas Personal Finance ha desarrollado una estrategia activa de apoyo a los minoristas, fabricantes y concesionarios de automóviles, comerciantes Web y diversas instituciones financieras (banca y seguros), basada en su experiencia en el mercado de crédito y su capacidad para ofrecer servicios adaptados a la actividad y la estrategia comercial de sus socios comerciales. También es un actor clave en materia de crédito responsable y de concienciación presupuestaria.

BNPP Personal Finance está, por naturaleza, expuesta al Riesgo de Crédito, y se basa en gran medida en modelos cuantitativos para gestionarlo. Dentro de BNP Paribas Personal Finance, el Departamento Central de Riesgos es responsable de la pertinencia de los modelos de calificación de riesgos utilizados en todas las entidades locales y de mantener un alto nivel de experiencia en la integración de nuevas técnicas estadísticas en nuevos entornos de modelización.

El equipo de Optimización de Procesos de Crédito forma parte del departamento de RIESGO de BNPP PF, dentro de Risk Personal Finance Global Credit Decision-making Policies, contribuimos a la racionalización y la optimización de los procesos de decisión de riesgo a través de un enfoque analítico. Apoyamos a los equipos de riesgo locales para mejorar la eficiencia de los procesos de crédito, incluida la parte de fraude, participando en el mejor equilibrio entre rentabilidad, recorrido del cliente y perfiles de riesgo.

**El fraude es un problema importante para los comerciantes. Los delincuentes utilizan una amplia variedad de métodos para atacar a las organizaciones a través de sistemas, canales, procesos y productos. Por ello, el desarrollo de métodos de detección del fraude reviste una importancia crucial. La detección del fraude es un problema difícil porque los defraudadores hacen todo lo posible para que su comportamiento parezca legítimo. Otra dificultad es que el número de registros legítimos es mucho mayor que el número de casos fraudulentos.**

**En nuestro caso, trabajeremos con un data ya pre-procesado para poder realizar el modelo de perceptron y regresión logística**

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

### Librerias

In [ ]:
# Data Analysis
import numpy as np
import pandas as pd

#Visualization
import matplotlib.pyplot as plt
import seaborn as sns

#Classificaon
from sklearn.linear_model import Perceptron , LogisticRegression

#Utils
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.model_selection import train_test_split

### Funciones de ayuda

In [ ]:
def high_correlation_features(dataframe, threshold=0.95):
    """
    Identifies and returns pairs of highly correlated features from the given dataframe.

    Parameters:
    - dataframe: A pandas DataFrame containing the dataset.
    - threshold: A float representing the correlation threshold to identify high correlations.

    Returns:
    - A DataFrame with pairs of features that have a correlation coefficient above the threshold.
    """
    # Calculate the correlation matrix
    corr_matrix = dataframe.corr()

    # Find features with a correlation above the threshold
    # Note: The matrix is symmetric, so we need to filter out one side to avoid duplicates
    high_corr_pairs = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
                       .stack()
                       .reset_index())
    high_corr_pairs.columns = ['Feature 1', 'Feature 2', 'Correlation']
    high_corr_pairs = high_corr_pairs.loc[high_corr_pairs['Correlation'] > threshold, :]

    return high_corr_pairs


In [ ]:
def remove_highly_correlated_features(dataframe, threshold=0.95):
    """
    Removes features that are highly correlated with each other above a specified threshold.

    Parameters:
    - dataframe: A pandas DataFrame containing the dataset.
    - threshold: A float representing the correlation threshold to identify high correlations.

    Returns:
    - A DataFrame with the highly correlated features removed.
    """
    # Calculate the correlation matrix
    corr_matrix = dataframe.corr().abs()
    
    # Select upper triangle of correlation matrix
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    
    # Find features with correlation greater than the threshold
    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > threshold)]
    
    # Drop features 
    reduced_df = dataframe.drop(to_drop, axis=1)
    
    return reduced_df, to_drop


In [ ]:
def plot_correlation_matrix(dataframe, plot_graph=True, return_matrix=False):
    """
    Calculates and optionally plots the correlation matrix of a given DataFrame.
    
    Parameters:
    - dataframe: DataFrame from which to calculate the correlation matrix.
    - plot_graph: If True, displays a heatmap of the correlation matrix.
    - return_matrix: If True, returns the correlation matrix.
    
    Returns:
    - If return_matrix is True, returns the correlation matrix of the dataframe.
    """
    # Calculate the correlation matrix
    correlation_matrix = dataframe.corr('spearman')
    
    # Plot the correlation matrix heatmap if requested
    if plot_graph:
        plt.figure(figsize=(25, 20))
        sns.heatmap(correlation_matrix, vmin=-1, vmax=1, center=0, cmap="hot", annot=True, fmt=".2f", square=True)
        plt.xticks(rotation=45, horizontalalignment='right')
    
    # Return the correlation matrix if requested
    if return_matrix:
        return correlation_matrix

In [ ]:
def plot_confusion_matrix(y_true, y_pred):
    """
    Plots a confusion matrix using Seaborn's heatmap.

    Parameters:
    - y_true: array-like of shape (n_samples,), True labels of the data.
    - y_pred: array-like of shape (n_samples,), Predicted labels.

    Returns:
    - None, displays a confusion matrix.
    """
    # Compute confusion matrix
    confusion_mat = confusion_matrix(y_true, y_pred)

    # Create a DataFrame for Seaborn's heatmap
    confusion_df = pd.DataFrame(confusion_mat, index=['Real Negative', 'Real Positive'], columns=['Predicted Negative', 'Predicted Positive'])
    
    # Plotting the heatmap
    plt.figure(figsize=(10,7))
    sns.heatmap(confusion_df, annot=True, fmt='g', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('Actual Values')
    plt.xlabel('Predicted Values')
    plt.show()

In [ ]:
def plot_roc_curve(clf, X_test, y_test, figsize=(10, 7)):
    """
    Plots the ROC curve by handling classifiers with or without the `predict_proba` method.
    Uses `decision_function` or binary predictions as fallbacks.
    
    Parameters:
    - clf: Classifier to evaluate.
    - X_test: Test data features.
    - y_test: True labels for the test data.
    - figsize: Size of the plot.
    """
    
    try:
        # First try to use predict_proba
        y_scores = clf.predict_proba(X_test)[:, 1]
    except AttributeError:
        try:
            # Next, try to use decision_function
            y_scores = clf.decision_function(X_test)
            # Convert decision scores to probabilities (min-max scaling)
            y_scores = (y_scores - y_scores.min()) / (y_scores.max() - y_scores.min())
        except AttributeError:
            # As a last resort, use binary predictions
            # This approach lacks precision and should be used cautiously
            y_pred = clf.predict(X_test)
            y_scores = np.where(y_pred == 1, 1, 0)  # Assuming the positive class is labeled as 1

    # Calculate ROC curve and AUC
    fpr, tpr, _ = roc_curve(y_test, y_scores)
    roc_auc = auc(fpr, tpr)
    
    # Plotting
    plt.figure(figsize=figsize)
    plt.plot(fpr, tpr, label=f"ROC curve (area = {roc_auc:.2f})")
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()




### Carga de Datos

In [ ]:
#df = pd.read_csv('./drive/MyDrive/Data/FraudeCanastas.csv') #Colab
df = pd.read_csv('./Data/FraudeCanastas.csv') #Colab

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.columns

In [ ]:
df

In [ ]:
df = df.set_index('ID')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
print(f'Los valores nulos son {sum(df.isnull().sum())}')

In [ ]:
# Distribution of the fraud flag
fraud_distribution = df['fraud_flag'].value_counts(normalize=True) * 100

In [ ]:
fraud_distribution

In [ ]:
df['fraud_flag'].plot(kind='hist')
plt.show()

Vamos a ver correlaciones entre las caracteristicas:

In [ ]:
#high_corr_features_df = high_correlation_features(df, 0.99)

In [ ]:
#reduced_df, _ = remove_highly_correlated_features(df, 0.99)

In [ ]:
#Separar las etiquetas del conjunto de datos
X = df.drop('fraud_flag', axis=1)
y = df['fraud_flag']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    shuffle=True,
                                                    random_state=42) 

In [ ]:
# imprime los conjuntos de entrenamiento y prueba
print("TRAINING SET")
print("X: ", X_train.shape)
print("y: ", y_train.shape)
print('Fraude:{:7.3f}%'.format(y_train.mean()*100),'\n')

print("TEST SET")
print("X: ", X_test.shape)
print("y: ", y_test.shape)
print('Fraude:{:7.3f}%'.format(y_test.mean()*100))

Vamos a entrenar un Perceptrón:

In [ ]:
model = Perceptron(max_iter=100, random_state=42, verbose=True) 

In [ ]:
model.fit(X_train,y_train)     

In [ ]:
y_pred = model.predict(X_test)

Veamos las métricas del modelo:

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
print("Precisión conjunto entrenamiento: %.2f%%" % (model.score(X_train, y_train)*100.0))
print("Precisión conjunto prueba: %.2f%%" % (model.score(X_test, y_test)*100.0))

In [ ]:
plot_confusion_matrix(y_test, y_pred)

In [ ]:
plot_roc_curve(model, X_test, y_test)

Veamos ahora la clasificación con Perceptron con margen:

In [ ]:
model_margin = Perceptron(penalty='l2', max_iter=100, random_state=42, verbose=True, alpha=0.00005) 

In [ ]:
model_margin.fit(X_train,y_train)     

In [ ]:
y_pred = model_margin.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
print("Precisión conjunto entrenamiento: %.2f%%" % (model_margin.score(X_train, y_train)*100.0))
print("Precisión conjunto prueba: %.2f%%" % (model_margin.score(X_test, y_test)*100.0))

In [ ]:
plot_confusion_matrix(y_test, y_pred)

In [ ]:
plot_roc_curve(model_margin, X_test, y_test)

In [ ]:
model_margin_1 = Perceptron(penalty='l2', max_iter=100, random_state=42, verbose=True, alpha=0.0005) 

In [ ]:
model_margin_1.fit(X_train,y_train)

In [ ]:
y_pred = model_margin_1.predict(X_test)

In [ ]:
plot_confusion_matrix(y_test, y_pred)

In [ ]:
plot_roc_curve(model_margin_1, X_test, y_test)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
print("Precisión conjunto entrenamiento: %.2f%%" % (model_margin_1.score(X_train, y_train)*100.0))
print("Precisión conjunto prueba: %.2f%%" % (model_margin_1.score(X_test, y_test)*100.0))

In [ ]:
model_margin_2 = Perceptron(penalty='l2', max_iter=100, random_state=42, verbose=True, alpha=0.005) 

In [ ]:
model_margin_2.fit(X_train,y_train)

In [ ]:
y_pred = model_margin_2.predict(X_test)

In [ ]:
plot_confusion_matrix(y_test, y_pred)

In [ ]:
plot_roc_curve(model_margin_2, X_test, y_test)

In [ ]:
print("Precisión conjunto entrenamiento: %.2f%%" % (model_margin_2.score(X_train, y_train)*100.0))
print("Precisión conjunto prueba: %.2f%%" % (model_margin_2.score(X_test, y_test)*100.0))

Ejercicios:

* Realizar feature selection

* Optimizar el modelo del Perceptron.  Optar por la regularización L1 o L2.

* Comenzar el challenge desde cero y realizar el prepocesamiento. No necesariamente tienen que llegar al mismo preprocesamiento usado. Pueden tomar sus decisiones de compromiso como así lo deseen.

![Lenguaje Matemático](./Images/Matematicas.png)

![Contacto](./Images/Contacto.png)